In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sess = tf.InteractiveSession()

In [3]:
weights = np.random.rand(3, 3, 256, 256)

In [4]:
w = tf.Variable(weights, tf.float32)

In [5]:
shape = w.get_shape().as_list()
shape

[3, 3, 256, 256]

In [6]:
w = tf.reshape(w, (shape[0], shape[1], shape[2] * shape[3]))
w

<tf.Tensor 'Reshape:0' shape=(3, 3, 65536) dtype=float64>

In [7]:
def get_gc(m):
    pow_2 = tf.constant(1.5, tf.float64)
    r = tf.reshape(m, [-1])
    size = r.get_shape().as_list()[0]
    # First moment: mean
    m1 = tf.reduce_mean(r)
    # Second Moment: standard deviation
    m2 = tf.div(tf.reduce_mean(tf.square(tf.sub(r, m1))), size)
    # Third moment
    m3 = tf.div(tf.reduce_mean(tf.pow((tf.sub(r, m1)), 3)), size)
    # Fourth Moment
    m4 = tf.div(tf.reduce_mean(tf.pow((tf.sub(r, m1)), 4)), size)
    # Skewness
    m5 = tf.pow(tf.div(m3, m2), 2)
    # Kurtosis
    m6 = tf.div(m4, tf.square(m2))
    gc = tf.pack([m1, m2, m3, m4, m5, m6])
    return gc

get_gc(w)

<tf.Tensor 'pack:0' shape=(6,) dtype=float64>

In [8]:
def get_hd(s_param, type):
    s = s_param
    row = s.get_shape().as_list()[0]
    col = s.get_shape().as_list()[1]
    dim1 = s.get_shape().as_list()[2]
    size = row

    if type == 1:
        size = row * col
        s = tf.reshape(s, [size, dim1])

    lst = []
    for i in range(0, size):
        gc = get_gc(s[i])
        lst.append(gc)
        
    g = tf.pack(lst)

    lst_2 = []
    for i in range(0, 6):
        size = g[i].get_shape().as_list()[0]
        # First momen: mean
        hd1 = tf.reduce_mean(g[i])
        # Second Moment: standard deviation
        hd2 = tf.div(tf.reduce_mean(tf.square(tf.sub(g[i], hd1))), size)
        # Third moment
        hd3 = tf.div(tf.reduce_mean(tf.pow((tf.sub(g[i], hd1)), 3)), size)
        # Fourth Moment
        hd4 = tf.div(tf.reduce_mean(tf.pow((tf.sub(g[i], hd1)), 4)), size)
        # Skewness
        hd5 = tf.pow(tf.div(hd3, hd2), 1.5)
        # Kurtosis
        hd6 = tf.div(hd4, tf.square(hd2))
        # Pack
        hd = tf.pack([hd1, hd2, hd3, hd4, hd5, hd6])
        lst_2.append(hd)
    h = tf.pack(lst_2)
    return h

In [9]:
# get_hd(w, 0)[:]

In [10]:
# get_hd(w, 1)[:]

In [11]:
# res[len(res) + 1] = get_hd(tf.transpose(w), 0)[:]
# This takes forever to run. It's problematic and causes our iterations to jump from 0.3s to 6s. 

In [12]:
res = {}

In [13]:
res[len(res) + 1] = get_gc(w)
#res[len(res) + 1] = get_hd(w, 0)[:]
res[len(res) + 1] = get_hd(w, 1)[:]
# len(res) + 1 
# This should give 4

In [14]:
state = tf.reshape(res[1], [6, 1])

In [15]:
for i in range(2, len(res) + 1):
    state = tf.concat(1, [state, res[i]])

st = tf.reshape(state, [-1])
st

<tf.Tensor 'Reshape_14:0' shape=(42,) dtype=float64>

In [16]:
sess.run(tf.initialize_all_variables())
wt = sess.run(w)
wt.shape

(3, 3, 65536)

In [17]:
st.eval()

array([  5.00151697e-01,   1.96175257e+04,   3.20702834e+08,
         2.51654564e+13,   2.59182775e+18,   2.19812821e+07,
         2.52000000e+01,   1.41295009e-07,   1.96099708e+04,
         3.20455867e+08,   2.51363928e+13,   2.58783746e+18,
         2.19685854e+07,   2.52000000e+01,   5.72964749e-11,
         1.96953697e+04,   3.23253028e+08,   2.54662218e+13,
         2.63321151e+18,   2.21122468e+07,   2.52000000e+01,
         2.12069257e-08,   1.96137573e+04,   3.20579623e+08,
         2.51509553e+13,   2.58983663e+18,   2.19749481e+07,
         2.52000000e+01,   1.64437992e-07,   1.96945220e+04,
         3.23225217e+08,   2.54629354e+13,   2.63275843e+18,
         2.21108200e+07,   2.52000000e+01,   1.06224348e+06,
         1.97263793e+04,   3.24271746e+08,   2.55867001e+13,
         2.64983456e+18,   2.21644906e+07,   2.52000000e+01])